In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from preprocessing import parse_aug_fn, parse_fn

In [2]:
valid_data = tfds.load('cifar10', split='train[:10%]')

train_data, info = tfds.load('cifar10', split='train[10%:100%]', with_info=True)

test_data = tfds.load('cifar10', split=tfds.Split.TEST)

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 64
train_num = int(info.splits['train'].num_examples / 10) * 9

train_data = train_data.shuffle(train_num)
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

valid_data = valid_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

test_data = test_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [4]:
def build_and_train_model(run_name, init):
    inputs = keras.Input(shape=(32, 32, 3))
    
    x = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=init)(inputs)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=init)(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer=init)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer=init)(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer=init)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu', kernel_initializer=init)(x)
    x = layers.Dropout(0.5)(x)
    
    outputs = layers.Dense(10, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    
    logfiles = 'lab5-logs\\{}-{}'.format(run_name, init.__class__.__name__)
    model_cbk = keras.callbacks.TensorBoard(log_dir=logfiles, histogram_freq=1)
    modelfiles = model_dir + '\\{}-best-model.h5'.format(run_name)
    model_mckp = keras.callbacks.ModelCheckpoint(modelfiles, monitor='val_categorical_accuracy', save_best_only=True, mode='max')
    
    model.compile(keras.optimizers.Adam(), loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.CategoricalAccuracy()])
    
    model.fit(train_data, epochs=100, validation_data=valid_data, callbacks=[model_cbk, model_mckp])

In [5]:
session_num = 1
model_dir = 'lab5-logs\\models\\'
os.makedirs(model_dir)

weights_initialization_list = [initializers.RandomNormal(0, 0.01), initializers.glorot_normal(), initializers.he_normal()]

for init in weights_initialization_list:
    print('---Running training session %d' % (session_num))
    run_name = 'run-%d' % session_num
    build_and_train_model(run_name, init)
    session_num += 1

---Running training session 1
Epoch 1/100


W0618 17:36:04.203359 13168 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


704/704 [==============================] - 60s 86ms/step - loss: 2.3028 - categorical_accuracy: 0.0986 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/100
704/704 [==============================] - 58s 82ms/step - loss: 2.3027 - categorical_accuracy: 0.0963 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 3/100
704/704 [==============================] - 60s 85ms/step - loss: 2.3027 - categorical_accuracy: 0.0974 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 4/100
704/704 [==============================] - 66s 94ms/step - loss: 2.3027 - categorical_accuracy: 0.0970 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 5/100
704/704 [==============================] - 78s 111ms/step - loss: 2.3027 - categorical_accuracy: 0.0967 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 6/100
704/704 [==============================] - 82s 117ms/step - loss: 2.3027 - categorical_accuracy: 0.0968 - val_loss: 2.3028 - val_categorical_accur

704/704 [==============================] - 88s 126ms/step - loss: 2.3027 - categorical_accuracy: 0.0968 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 50/100
704/704 [==============================] - 89s 127ms/step - loss: 2.3027 - categorical_accuracy: 0.0968 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 51/100
704/704 [==============================] - 91s 129ms/step - loss: 2.3027 - categorical_accuracy: 0.0968 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 52/100
704/704 [==============================] - 91s 130ms/step - loss: 2.3027 - categorical_accuracy: 0.0968 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 53/100
704/704 [==============================] - 92s 130ms/step - loss: 2.3027 - categorical_accuracy: 0.0968 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 54/100
704/704 [==============================] - 93s 133ms/step - loss: 2.3027 - categorical_accuracy: 0.0968 - val_loss: 2.3028 - val_categorical_accu

Epoch 97/100
704/704 [==============================] - 94s 133ms/step - loss: 2.3027 - categorical_accuracy: 0.0968 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 98/100
704/704 [==============================] - 109s 155ms/step - loss: 2.3027 - categorical_accuracy: 0.0968 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 99/100
704/704 [==============================] - 94s 133ms/step - loss: 2.3027 - categorical_accuracy: 0.0968 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
Epoch 100/100
704/704 [==============================] - 93s 133ms/step - loss: 2.3027 - categorical_accuracy: 0.0968 - val_loss: 2.3028 - val_categorical_accuracy: 0.0952
---Running training session 2
Epoch 1/100
704/704 [==============================] - 95s 135ms/step - loss: 2.0551 - categorical_accuracy: 0.2291 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/100
704/704 [==============================] - 107s 152ms/step - loss: 1.8358 - categorical_accura

704/704 [==============================] - 104s 147ms/step - loss: 0.9672 - categorical_accuracy: 0.6729 - val_loss: 0.9509 - val_categorical_accuracy: 0.6784
Epoch 45/100
704/704 [==============================] - 108s 154ms/step - loss: 0.9674 - categorical_accuracy: 0.6728 - val_loss: 0.9538 - val_categorical_accuracy: 0.6706
Epoch 46/100
704/704 [==============================] - 94s 134ms/step - loss: 0.9597 - categorical_accuracy: 0.6766 - val_loss: 0.9399 - val_categorical_accuracy: 0.6842
Epoch 47/100
704/704 [==============================] - 110s 156ms/step - loss: 0.9599 - categorical_accuracy: 0.6728 - val_loss: 0.9509 - val_categorical_accuracy: 0.6858
Epoch 48/100
704/704 [==============================] - 110s 157ms/step - loss: 0.9481 - categorical_accuracy: 0.6785 - val_loss: 1.0015 - val_categorical_accuracy: 0.6620
Epoch 49/100
704/704 [==============================] - 109s 155ms/step - loss: 0.9452 - categorical_accuracy: 0.6799 - val_loss: 0.9691 - val_categorical

Epoch 92/100
704/704 [==============================] - 94s 133ms/step - loss: 0.8223 - categorical_accuracy: 0.7238 - val_loss: 0.8889 - val_categorical_accuracy: 0.7078
Epoch 93/100
704/704 [==============================] - 94s 133ms/step - loss: 0.8193 - categorical_accuracy: 0.7282 - val_loss: 0.8847 - val_categorical_accuracy: 0.7064
Epoch 94/100
704/704 [==============================] - 93s 133ms/step - loss: 0.8161 - categorical_accuracy: 0.7298 - val_loss: 0.9095 - val_categorical_accuracy: 0.7028
Epoch 95/100
704/704 [==============================] - 94s 133ms/step - loss: 0.8159 - categorical_accuracy: 0.7283 - val_loss: 0.8885 - val_categorical_accuracy: 0.7100
Epoch 96/100
704/704 [==============================] - 93s 133ms/step - loss: 0.8091 - categorical_accuracy: 0.7313 - val_loss: 0.9092 - val_categorical_accuracy: 0.6920
Epoch 97/100
704/704 [==============================] - 94s 133ms/step - loss: 0.8116 - categorical_accuracy: 0.7308 - val_loss: 0.8383 - val_cat

Epoch 40/100
704/704 [==============================] - 106s 151ms/step - loss: 0.8781 - categorical_accuracy: 0.7080 - val_loss: 0.8708 - val_categorical_accuracy: 0.7138
Epoch 41/100
704/704 [==============================] - 105s 149ms/step - loss: 0.8741 - categorical_accuracy: 0.7088 - val_loss: 0.8939 - val_categorical_accuracy: 0.7042
Epoch 42/100
704/704 [==============================] - 94s 133ms/step - loss: 0.8634 - categorical_accuracy: 0.7128 - val_loss: 0.8995 - val_categorical_accuracy: 0.7028
Epoch 43/100
704/704 [==============================] - 94s 133ms/step - loss: 0.8714 - categorical_accuracy: 0.7111 - val_loss: 0.8749 - val_categorical_accuracy: 0.7088
Epoch 44/100
704/704 [==============================] - 94s 133ms/step - loss: 0.8574 - categorical_accuracy: 0.7137 - val_loss: 0.8932 - val_categorical_accuracy: 0.6986
Epoch 45/100
704/704 [==============================] - 94s 133ms/step - loss: 0.8517 - categorical_accuracy: 0.7171 - val_loss: 0.9356 - val_c

Epoch 88/100
704/704 [==============================] - 93s 132ms/step - loss: 0.7364 - categorical_accuracy: 0.7565 - val_loss: 0.8465 - val_categorical_accuracy: 0.7228
Epoch 89/100
704/704 [==============================] - 93s 131ms/step - loss: 0.7294 - categorical_accuracy: 0.7603 - val_loss: 0.8337 - val_categorical_accuracy: 0.7302
Epoch 90/100
704/704 [==============================] - 93s 132ms/step - loss: 0.7374 - categorical_accuracy: 0.7577 - val_loss: 0.8539 - val_categorical_accuracy: 0.7340
Epoch 91/100
704/704 [==============================] - 93s 132ms/step - loss: 0.7399 - categorical_accuracy: 0.7584 - val_loss: 0.8327 - val_categorical_accuracy: 0.7220
Epoch 92/100
704/704 [==============================] - 93s 132ms/step - loss: 0.7380 - categorical_accuracy: 0.7572 - val_loss: 0.8898 - val_categorical_accuracy: 0.7158
Epoch 93/100
704/704 [==============================] - 94s 134ms/step - loss: 0.7354 - categorical_accuracy: 0.7580 - val_loss: 0.8214 - val_cat

In [6]:
model_1 = keras.models.load_model('lab5-logs/models/run-1-best-model.h5')
model_2 = keras.models.load_model('lab5-logs/models/run-2-best-model.h5')
model_3 = keras.models.load_model('lab5-logs/models/run-3-best-model.h5')

In [7]:
loss_1, acc_1 = model_1.evaluate(test_data)
loss_2, acc_2 = model_2.evaluate(test_data)
loss_3, acc_3 = model_3.evaluate(test_data)

    157/Unknown - 5s 32ms/step - loss: 0.8324 - categorical_accuracy: 0.7331

In [9]:
loss = [loss_1, loss_2, loss_3]
acc = [acc_1, acc_2, acc_3]
dict = {'Loss': loss, 'Accuracy': acc}
pd.DataFrame(dict)

,Loss,Accuracy
0,2.302636,0.1000
1,0.871096,0.7162
2,0.832405,0.7331
